In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
def input_preprocess(x, hflip=True):
    x = x.float()  # input data (integer 0 ~ 255)
    x = x / 255.0  # 0~1사이의 float로 바꿔주기
    if hflip:
        half = int(x.shape[0] / 2)  # mini-batch의 절반의 크기를 계산
        x[0:half, :] = torch.flip(
            x[0:half, :], dims=[2]
        )  # 앞에서부터 절반까지는 수평뒤집기(좌우반전)

    x = torch.permute(x, dims=[0, 3, 1, 2])  # 차원의 순서를 바꿔주기 B,H,W,C -> B,C,H,W
    return x

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=32, kernel_size=(3, 3), padding=0, stride=(1, 1)
        )
        self.conv2 = nn.Conv2d(32, 64, (4, 4))
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(64 * 6 * 6, 10)

    def forward(self, x):  # 3x32x32
        x = self.conv1(x)  # 32x30x30
        x = F.relu(x)
        x = F.max_pool2d(x, 2)  # 32x15x15

        x = self.conv2(x)  # 64x12x12
        x = F.relu(x)
        x = F.max_pool2d(x, 2)  # 64x6x6

        x = torch.flatten(
            x, start_dim=1
        )  # Fully Connected Layer를 통과할 수 있도록 쭉 펼침
        x = self.dropout(x)
        x = self.fc1(x)

        return x
